In [2]:
import os, sys, time
import pandas as pd
import numpy as np

In [3]:
# Set up core table from Yahoo projection
stats = pd.DataFrame.from_csv('yahoo_proj_2016.csv').reset_index()
stats['Name'] = stats['Players'].str.rsplit(' ',n=2).str.get(0)
stats['TON'] = -1*stats['AST']/stats['A/T']
stats['REB'] = stats['OREB'] + stats['DREB']
cats = ['GP','FGM','FGA','FTM','FTA','3PTM','AST','REB','ST','BLK','TON']
stats = stats[['Name','Pre'] + cats].sort_values(by='Pre')
stats['Owner'] = np.nan
stats['Dround'] = np.nan
stats.set_index('Name', inplace=True)
stats = stats[0:300]
final = stats.copy()

In [4]:
# Set up teams
teams = ['A','B','C','D','E','F','G','H','I','J','K','L']
rosters = ['PG','SG','G','SF','PF','F','C','C','Util','Util','BN','BN','BN']

def select(core, team, dround):
    picked_name = np.nan
    
    if dround==1:
        # First round, always follow rank
        picked_name = core[core.Owner.isnull()].sort_values(by='Pre').iloc[0].name
    else:
        # Second round on, select player who maximizes per-player-per-game-avg ranking
        picked_name = select_max_3pga_fast(core, team)

    return picked_name 

def compute_3pga(core):
    ''' Compute Per-Player-Per-Game-Averages'''
    grouped = core.groupby('Owner')
    standing = grouped.mean()[cats]
    standing['Count'] = grouped.size()
    return standing

def compute_ranking(standing):
    ''' From 3PGA compute ranking and fantasy point'''
    tmp = standing
    tmp['FGP']=tmp['FGM']/tmp['FGA']
    tmp['FTP']=tmp['FTM']/tmp['FTA']
    tmp = tmp[['FGP','FTP','3PTM','REB','AST','ST','BLK','TON']]
    tmp = tmp.rank()
    tmp['FP'] = tmp.sum(axis=1)
    tmp.sort_values(by='FP', ascending=False, inplace=True)
    return tmp

def select_max_3pga(core, team):
    reach = 12
    cands = core[core.Owner.isnull()].ix[0:reach]
    cands['Owner'] = team
    fp_cands = []
    for ii in range(0, reach):
        rank_cand = compute_ranking(compute_3pga(core[~core.Owner.isnull()].append(cands.ix[ii])))
        fp_cands.append(rank_cand.ix[team].FP)
    return cands.ix[np.argmax(fp_cands)].name

def select_max_3pga_fast(core, team):
    reach = 12
    cands = core[core.Owner.isnull()].ix[0:reach]
    cands['Owner'] = team
    fp_cands = []
    current_3pga = compute_3pga(core[~core.Owner.isnull()])
    for ii in range(0, reach):
        team_new_3pga = compute_3pga(core[~core.Owner.isnull() & (core.Owner==team)].append(cands.ix[ii]))
        current_3pga.loc[team] = team_new_3pga.loc[team]
        rank_cand = compute_ranking(current_3pga)
        fp_cands.append(rank_cand.ix[team].FP)
    return cands.ix[np.argmax(fp_cands)].name

In [5]:
#Start

def run_draft(final, teams, rosters, num_picks):
    N = len(teams)
    M = len(rosters)
    core = final.copy()
    for s in range(0, num_picks):
        
        # Init parameters for this pick
        dround = s/N+1
        team = teams[s%N if s/N%2==0 else N-s%N-1]
        pick = s+1

        # Make pick 
        picked_name = select(core, team, dround)
        core.ix[picked_name, 'Owner'] = team
        core.ix[picked_name, 'Dround'] = dround
        core.ix[picked_name, 'Pick'] = pick
        
        #print "Team {0} picks for round {1}: {2}".format(team, dround, picked_name)
    return core

In [6]:
t0 = time.time()
core_run = run_draft(final, teams, rosters, 12*12)
print time.time()-t0

47.3012650013


In [7]:
core_run.sort_values(by='Pick')
ranking_run = compute_ranking(compute_3pga(core_run))

In [8]:
ranking_run

,FGP,FTP,3PTM,REB,AST,ST,BLK,TON,FP
Owner,,,,,,,,,
B,8.0,7.0,7.0,7.0,12.0,8.0,6.0,3.0,58.0
H,10.0,1.0,4.0,12.0,6.0,5.0,12.0,7.0,57.0
A,3.0,12.0,12.0,1.0,10.0,11.0,1.0,6.0,56.0
F,2.0,9.0,10.0,3.0,5.0,12.0,3.0,9.0,53.0
E,12.0,3.0,1.0,11.0,1.0,3.0,11.0,10.0,52.0
I,1.0,11.0,11.0,2.0,11.0,9.0,2.0,5.0,52.0
K,11.0,6.0,2.0,10.0,2.0,1.0,8.0,12.0,52.0
L,9.0,2.0,6.0,9.0,3.0,2.0,9.0,11.0,51.0
C,7.0,8.0,9.0,4.0,7.0,6.0,5.0,4.0,50.0


In [9]:
from IPython.display import display, HTML
out = core_run[~core_run.Owner.isnull()][['Owner','Pick','Pre']].sort_values(['Owner', 'Pick'])
out['Reach'] = out['Pre'] - out['Pick']
HTML(out.to_html())

,Owner,Pick,Pre,Reach
Name,,,,
S. Curry GS,A,1.0,1,0.0
J. Butler Chi,A,24.0,16,-8.0
D. Favors Uta,A,25.0,28,3.0
I. Thomas Bos,A,48.0,37,-11.0
D. Collison Sac,A,49.0,56,7.0
G. Harris Den,A,72.0,82,10.0
D. Carroll Tor,A,73.0,84,11.0
J. Clarkson LAL,A,96.0,83,-13.0
C. Bosh Mia,A,97.0,105,8.0


In [210]:
core_run[0:100]

,Pre,GP,FGM,FGA,FTM,FTA,3PTM,AST,REB,ST,BLK,TON,Owner,Dround,Pick
Name,,,,,,,,,,,,,,,
S. Curry GS,1,78,9.0,18.3,4.5,4.9,4.7,7.0,4.7,2.0,0.2,-3.181818,A,1.0,1.0
K. Durant GS,2,77,8.9,17.5,5.8,6.6,2.8,4.5,7.5,1.0,1.2,-3.214286,B,1.0,2.0
J. Harden Hou,3,80,8.5,19.0,8.9,10.3,2.8,7.3,5.8,1.8,0.5,-4.294118,C,1.0,3.0
R. Westbrook OKC,4,80,9.4,21.3,7.7,9.3,1.4,9.5,8.0,2.1,0.3,-4.523810,D,1.0,4.0
K. Towns Min,5,81,8.4,15.6,3.2,3.9,0.5,2.1,11.1,0.8,1.7,-2.625000,E,1.0,5.0
K. Leonard SA,6,71,7.8,15.8,4.1,4.8,1.8,2.8,6.7,1.9,0.9,-1.647059,F,1.0,6.0
G. Antetokounmpo Mil,7,80,7.3,14.4,4.1,5.6,0.4,6.8,8.3,1.4,1.5,-2.956522,G,1.0,7.0
A. Davis NO,8,69,9.4,18.6,5.5,7.0,0.8,2.2,10.2,1.3,2.4,-2.000000,H,1.0,8.0
C. Paul LAC,9,73,6.6,14.0,3.6,4.1,1.6,10.0,4.1,2.1,0.2,-2.380952,I,1.0,9.0
